<a href="https://colab.research.google.com/github/dakilaledesma/272-iNat-Final/blob/main/%5BFSUS%5D_S2D_O600G4_FullGrad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%time
! unzip -q drive/MyDrive/UNC/S2022P/GBIF_O_21_fourth.zip -d /content/temp
! mv /content/temp/content/dataset/test /content/test
! rm -rf /content/temp

CPU times: user 48.1 ms, sys: 21.7 ms, total: 69.8 ms
Wall time: 5.9 s


In [ ]:
! pip install timm grad-cam tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

import matplotlib.pyplot as plt
from PIL import Image
import torch.nn as nn
import numpy as np
import os, json
from glob import glob
from tqdm.notebook import tqdm

import torch
from torchvision import models, transforms
from torch.autograd import Variable
import torch.nn.functional as F

import timm

In [ ]:
torch.backends.cudnn.enabled = False

In [ ]:
def get_input_transform():
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])       
    transf = transforms.Compose([
        transforms.Resize((600, 600)),
        transforms.ToTensor(),
        normalize
    ])    

    return transf

def get_input_tensors(img):
    transf = get_input_transform()
    return transf(img).unsqueeze(0)

In [ ]:
ckpt_fn = "drive/MyDrive/UNC/S2022P/output/20220816-163320-sequencer2d_l-600/last.pth.tar"
im_path = "test"
out_path = "resolution/600_s2d"
model_type = "sequencer2d_l"

fl_dict = {}
for idx, fl in enumerate(glob(f"{im_path}/*")):
    bn = os.path.basename(fl)
    fl_dict[bn] = idx
    
images = []
for im_fn in glob(f"{im_path}/**/*.*", recursive=True):
    bn = os.path.basename(im_fn)
    images.append(im_fn)
    
print(len(images))

841


In [ ]:
model = timm.create_model(
        model_type,
        num_classes=185,
        in_chans=3,
        checkpoint_path=ckpt_fn)

In [ ]:
for im_fn in tqdm(images, total=len(list(images))):
    curr_dir = dir()
    cam_list = [["fullgrad", FullGrad(model=model, target_layers=[], use_cuda=True)]]
    
    bn = os.path.basename(im_fn)
    cat = int(im_fn.split("/")[-2])
    
    img = Image.open(im_fn)
    img_t = get_input_tensors(img)
    test_img = np.array(img.resize((600, 600)), dtype='float32')
    test_img /= 255

    targets = [ClassifierOutputTarget(cat)]
    
    for cam_str, cam_method in cam_list:
        if not os.path.exists(f"outs/{out_path}/{cam_str}/{cat}_{bn}"):
            grayscale_cam = cam_method(input_tensor=img_t, targets=targets)
            grayscale_cam = grayscale_cam[0, :]

            visualization = show_cam_on_image(test_img, grayscale_cam, use_rgb=True)
            plt.imshow(visualization)

            try:
                plt.savefig(f"outs/{out_path}/{cam_str}/{cat}_{bn}")
            except FileNotFoundError:
                os.makedirs(f"outs/{out_path}/{cam_str}/")
            finally:
                plt.savefig(f"outs/{out_path}/{cam_str}/{cat}_{bn}")
              
            plt.close()
        
            del grayscale_cam
        
    for name in dir():
        if name not in curr_dir and name != "curr_dir":
            del globals()[name]

  0%|          | 0/841 [00:00<?, ?it/s]

In [ ]:
! zip -q -r -0 s2d_outs.zip outs/

In [ ]:
! mv s2d_outs.zip /content/drive/MyDrive/UNC/S2022P/o600g4_s2d_outs.zip